In [3]:
# MIE1621 Fall 2017
# Project 

import numpy as np
import matplotlib.pyplot as plt


def pprint(obj):
    '''
    For debugging, print statements with numpy variable names and shape
    '''
    def namestr(obj):
        namespace = globals()
        return [name for name in namespace if namespace[name] is obj]
    # Assumes obj is a numpy array, matrix
    try:
        print(namestr(obj), obj.shape)
    except:
        try:
            print(namestr(obj), ",", len(obj))
        except:
            print(namestr(obj))
    print(obj)


covarianceMatrix = np.array([[0.02778, 0.00387, 0.00021],
                             [0.00387, 0.01112, -0.00020],
                             [0.00021, -0.00020, 0.00115]])

expectedReturn = np.array([0.1073, 0.0737, 0.0627])

stepSize = 1.0

delta = 4.0

# Randomly initialize to anything
initialX = np.array([0.25, 0.25, 0.5])
initialPi = 1.2

def checkShape(x, u, sigma, delta):
    if delta <= 0.0:
        raise ValueError("Risk aversion, delta needs to be > 0")
    if delta <= 3.5 or delta >= 4.5:
        raise ValueError("Risk aversion, delta needs to be between (3.5, 4.5)")
    if sigma.shape[0] != sigma.shape[1]:
        raise ValueError("Covariance matrix must be square")
    if sigma.shape[0] != x.shape[0]:
        raise ValueError("Dimensions dont match between sigma and x")
    if u.shape[0] != x.shape[0]:
        raise ValueError("Dimensions dont match between u and x")
    '''
    Not always met, only met after training is over
    if np.sum(x) != 1.0:
        raise ValueError("X must sum to 1.0")
    '''

def computeF(x, u, sigma, delta):
    '''
    x is the proportion, the parameters we are changing to maximize f
    u is the expected return
    sigma is the covariance matrix
    delta is the parameter that controls risk aversion, delta > 0
    '''
    checkShape(x, u, sigma, delta)
    
    firstTerm = 0.0
    for (ui, xi) in zip(u, x):
        firstTerm += ui * xi
    secondTerm = 0.0
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            secondTerm += sigma[i][j] * x[i] * x[j]
    secondTerm *= (float(delta)/float(2.0))
            
    f = firstTerm - secondTerm
    return f

def computeGrad(x, pi, u, sigma, delta):
    checkShape(x, u, sigma, delta)
    grad = np.zeros((x.shape[0] + 1, 1))
    dLdx = np.zeros(x.shape)
    for i in range(x.shape[0]): 
        secondIndex = (i + 1) % x.shape[0]
        thirdIndex = (i + 2) % x.shape[0]
        
        dLdx[i] = u[i] + pi - ((delta)*((float(sigma[i][i]*x[i])/2.0) \
                                        + sigma[i][secondIndex]*x[secondIndex]) \
                               + sigma[i][thirdIndex] * x[thirdIndex])
        grad[i] = dLdx[i]
        
    dLdPi = np.sum(x) - 1.0
    grad[x.shape[0]] = dLdPi
    
    return grad; 

def computeHessian(x, pi, u, sigma, delta):
    checkShape(x, u, sigma, delta)
    dim = x.shape[0] + 1
    hessian = np.zeros((dim, dim))
    for i in range(x.shape[0]): 
        for j in range(x.shape[0]): 
            hessian[i][j] = (-1.0) * delta * sigma[i][j]
            if i == j:
                hessian[i][j] /= 2.0
    for i in range(x.shape[0]): 
        hessian[x.shape[0]][i] = 1.0
        hessian[i][x.shape[0]] = 1.0
    return hessian

x = initialX.copy()
pi = initialPi 
f = computeF(x, expectedReturn, covarianceMatrix, delta)

pprint(covarianceMatrix)
pprint(expectedReturn)
pprint(x)
pprint(f)

grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
pprint(grad)

hessian = computeHessian(x, pi, expectedReturn, covarianceMatrix, delta)
pprint(hessian)

maxIteration = 1000

['covarianceMatrix'] (3, 3)
[[ 0.02778  0.00387  0.00021]
 [ 0.00387  0.01112 -0.0002 ]
 [ 0.00021 -0.0002   0.00115]]
['expectedReturn'] (3,)
[ 0.1073  0.0737  0.0627]
['x'] (3,)
[ 0.25  0.25  0.5 ]
['f'] ()
0.07019
['grad'] (4, 1)
[[ 1.289435 ]
 [ 1.2675725]
 [ 1.26139  ]
 [ 0.       ]]
['hessian'] (4, 4)
[[ -5.55600000e-02  -1.54800000e-02  -8.40000000e-04   1.00000000e+00]
 [ -1.54800000e-02  -2.22400000e-02   8.00000000e-04   1.00000000e+00]
 [ -8.40000000e-04   8.00000000e-04  -2.30000000e-03   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00   0.00000000e+00]]


In [ ]:
# Newton Method
currX = x.copy()
currPi = pi
for iteration in range(maxIteration):
    pprint(iteration)
    
    x = currX.copy()
    pi = currPi
    
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(f)
    pprint(constraint)
    pprint(x)
    pprint(pi)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    hessian = computeHessian(x, pi, expectedReturn, covarianceMatrix, delta)
    
    direction = np.dot(np.linalg.inv(hessian), grad)
    pprint(direction)
    
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    
    # You always deduct in Newton Method!
    # As newton method accounts for correct direction regardless of max or min.
    # Newton's method always searches for critical points. 
    currX -= stepSize * xAdd
    currPi -= stepSize * direction[-1]

In [4]:
# Steepest Descent Method
# Note: NOT really steepest as the question asks to use stepsize of 1.0, so don't calculate stepsize. 
# Just do the normal gradient descent in the Machine Learning literature

currX = x.copy()
currPi = pi
for iteration in range(maxIteration):
    pprint(iteration)
    
    x = currX.copy()
    pi = currPi
    
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(f)
    pprint(constraint)
    pprint(x)
    pprint(pi)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    # No hessian computation needed for gradient descent
    
    direction = grad
    pprint(direction)
    
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    
    # Gradient Descent diverges with constant step size
    # stepSize = 0.00001 , can only converge with small step size but not OPTIMAL
    # Need to add cause maximizing the function, NOT minimizing
    currX += stepSize * xAdd
    currPi += stepSize * direction[-1]


['iteration']
0
['f'] ()
0.07019
['constraint'] ()
1.0
['x'] (3,)
[ 0.25  0.25  0.5 ]
['initialPi', 'pi', 'currPi']
1.2
['grad', 'direction'] (4, 1)
[[ 1.289435 ]
 [ 1.2675725]
 [ 1.26139  ]
 [ 0.       ]]
['iteration']
1
['f'] ()
0.0701924810418
['constraint'] ()
1.00003818398
['x'] (3,)
[ 0.25001289  0.25001268  0.50001261]
['pi', 'currPi'] (1,)
[ 1.2]
['grad', 'direction'] (4, 1)
[[  1.28943408e+00]
 [  1.26757218e+00]
 [  1.26138996e+00]
 [  3.81839750e-05]]
['iteration']
2
['f'] ()
0.0701949620512
['constraint'] ()
1.00007636794
['x'] (3,)
[ 0.25002579  0.25002535  0.50002523]
['pi', 'currPi'] (1,)
[ 1.2]
['grad', 'direction'] (4, 1)
[[  1.28943317e+00]
 [  1.26757186e+00]
 [  1.26138993e+00]
 [  7.63679373e-05]]
['iteration']
3
['f'] ()
0.0701974430283
['constraint'] ()
1.00011455189
['x'] (3,)
[ 0.25003868  0.25003803  0.50003784]
['pi', 'currPi'] (1,)
[ 1.2]
['grad', 'direction'] (4, 1)
[[  1.28943226e+00]
 [  1.26757154e+00]
 [  1.26138989e+00]
 [  1.14551887e-04]]
['iteration

['grad', 'direction'] (4, 1)
[[ 1.28931403]
 [ 1.26753225]
 [ 1.26138843]
 [ 0.00519291]]
['iteration']
137
['f'] ()
0.0705296018413
['constraint'] ()
1.00523109067
['x'] (3,)
[ 0.25176644  0.25173655  0.5017281 ]
['pi', 'currPi'] (1,)
[ 1.20000356]
['grad', 'direction'] (4, 1)
[[ 1.28931317]
 [ 1.26753198]
 [ 1.26138845]
 [ 0.00523109]]
['iteration']
138
['f'] ()
0.0705320784609
['constraint'] ()
1.005269273
['x'] (3,)
[ 0.25177934  0.25174922  0.50174072]
['pi', 'currPi'] (1,)
[ 1.20000361]
['grad', 'direction'] (4, 1)
[[ 1.28931231]
 [ 1.26753171]
 [ 1.26138846]
 [ 0.00526927]]
['iteration']
139
['f'] ()
0.0705345550482
['constraint'] ()
1.00530745533
['x'] (3,)
[ 0.25179223  0.2517619   0.50175333]
['pi', 'currPi'] (1,)
[ 1.20000366]
['grad', 'direction'] (4, 1)
[[ 1.28931144]
 [ 1.26753144]
 [ 1.26138848]
 [ 0.00530746]]
['iteration']
140
['f'] ()
0.0705370316033
['constraint'] ()
1.00534563764
['x'] (3,)
[ 0.25180512  0.25177457  0.50176594]
['pi', 'currPi'] (1,)
[ 1.20000372]
['

324
['f'] ()
0.0709921701352
['constraint'] ()
1.01237100548
['x'] (3,)
[ 0.25417731  0.25410679  0.50408691]
['pi', 'currPi'] (1,)
[ 1.20001998]
['grad', 'direction'] (4, 1)
[[ 1.28915846]
 [ 1.26748825]
 [ 1.26139789]
 [ 0.01237101]]
['iteration']
325
['f'] ()
0.0709946407426
['constraint'] ()
1.01240918592
['x'] (3,)
[ 0.2541902   0.25411946  0.50409952]
['pi', 'currPi'] (1,)
[ 1.2000201]
['grad', 'direction'] (4, 1)
[[ 1.28915767]
 [ 1.26748805]
 [ 1.26139798]
 [ 0.01240919]]
['iteration']
326
['f'] ()
0.0709971113178
['constraint'] ()
1.01244736636
['x'] (3,)
[ 0.2542031   0.25413214  0.50411213]
['pi', 'currPi'] (1,)
[ 1.20002023]
['grad', 'direction'] (4, 1)
[[ 1.28915687]
 [ 1.26748785]
 [ 1.26139807]
 [ 0.01244737]]
['iteration']
327
['f'] ()
0.0709995818611
['constraint'] ()
1.01248554679
['x'] (3,)
[ 0.25421599  0.25414481  0.50412475]
['pi', 'currPi'] (1,)
[ 1.20002035]
['grad', 'direction'] (4, 1)
[[ 1.28915608]
 [ 1.26748766]
 [ 1.26139815]
 [ 0.01248555]]
['iteration']
3

['x'] (3,)
[ 0.25671681  0.25660371  0.50657188]
['pi', 'currPi'] (1,)
[ 1.20005172]
['grad', 'direction'] (4, 1)
[[ 1.28900993]
 [ 1.26745662]
 [ 1.26142228]
 [ 0.01989239]]
['iteration']
522
['f'] ()
0.0714807257947
['constraint'] ()
1.01993057227
['x'] (3,)
[ 0.2567297   0.25661638  0.5065845 ]
['pi', 'currPi'] (1,)
[ 1.20005192]
['grad', 'direction'] (4, 1)
[[ 1.28900921]
 [ 1.2674565 ]
 [ 1.26142245]
 [ 0.01993057]]
['iteration']
523
['f'] ()
0.0714831900661
['constraint'] ()
1.01996875115
['x'] (3,)
[ 0.25674259  0.25662906  0.50659711]
['pi', 'currPi'] (1,)
[ 1.20005212]
['grad', 'direction'] (4, 1)
[[ 1.2890085 ]
 [ 1.26745637]
 [ 1.26142261]
 [ 0.01996875]]
['iteration']
524
['f'] ()
0.0714856543056
['constraint'] ()
1.02000693003
['x'] (3,)
[ 0.25675548  0.25664173  0.50660972]
['pi', 'currPi'] (1,)
[ 1.20005232]
['grad', 'direction'] (4, 1)
[[ 1.28900778]
 [ 1.26745625]
 [ 1.26142277]
 [ 0.02000693]]
['iteration']
525
['f'] ()
0.0714881185131
['constraint'] ()
1.0200451089
[

['pi', 'currPi'] (1,)
[ 1.20009828]
['grad', 'direction'] (4, 1)
[[ 1.28887624]
 [ 1.26743981]
 [ 1.26146149]
 [ 0.02741352]]
['iteration']
719
['f'] ()
0.0719655719377
['constraint'] ()
1.02745169579
['x'] (3,)
[ 0.25926891  0.25911325  0.50906953]
['pi', 'currPi'] (1,)
[ 1.20009855]
['grad', 'direction'] (4, 1)
[[ 1.2888756 ]
 [ 1.26743976]
 [ 1.26146173]
 [ 0.0274517 ]]
['iteration']
720
['f'] ()
0.0719680299353
['constraint'] ()
1.02748987356
['x'] (3,)
[ 0.2592818   0.25912593  0.50908215]
['pi', 'currPi'] (1,)
[ 1.20009883]
['grad', 'direction'] (4, 1)
[[ 1.28887496]
 [ 1.26743971]
 [ 1.26146197]
 [ 0.02748987]]
['iteration']
721
['f'] ()
0.0719704879012
['constraint'] ()
1.02752805133
['x'] (3,)
[ 0.25929469  0.2591386   0.50909476]
['pi', 'currPi'] (1,)
[ 1.2000991]
['grad', 'direction'] (4, 1)
[[ 1.28887432]
 [ 1.26743967]
 [ 1.26146221]
 [ 0.02752805]]
['iteration']
722
['f'] ()
0.0719729458353
['constraint'] ()
1.02756622909
['x'] (3,)
[ 0.25930758  0.25915127  0.50910738]
[

['f'] ()
0.072432021919
['constraint'] ()
1.03470540429
['x'] (3,)
[ 0.26171766  0.26152138  0.51146636]
['pi', 'currPi'] (1,)
[ 1.20015757]
['grad', 'direction'] (4, 1)
[[ 1.28876077]
 [ 1.26743766]
 [ 1.26151366]
 [ 0.0347054 ]]
['iteration']
910
['f'] ()
0.0724344738941
['constraint'] ()
1.03474358141
['x'] (3,)
[ 0.26173055  0.26153406  0.51147897]
['pi', 'currPi'] (1,)
[ 1.20015791]
['grad', 'direction'] (4, 1)
[[ 1.28876021]
 [ 1.26743768]
 [ 1.26151397]
 [ 0.03474358]]
['iteration']
911
['f'] ()
0.0724369258375
['constraint'] ()
1.03478175853
['x'] (3,)
[ 0.26174344  0.26154673  0.51149159]
['pi', 'currPi'] (1,)
[ 1.20015826]
['grad', 'direction'] (4, 1)
[[ 1.28875964]
 [ 1.26743771]
 [ 1.26151428]
 [ 0.03478176]]
['iteration']
912
['f'] ()
0.0724393777494
['constraint'] ()
1.03481993564
['x'] (3,)
[ 0.26175633  0.26155941  0.5115042 ]
['pi', 'currPi'] (1,)
[ 1.20015861]
['grad', 'direction'] (4, 1)
[[ 1.28875907]
 [ 1.26743773]
 [ 1.26151459]
 [ 0.03481994]]
['iteration']
913
[

In [ ]:
# Quasi Newton Methods 
# BFGS

H = np.eye(x.shape[0] + 1)

def BFGSUpdate(H, y, s): 
    Hnext = H + (1.0/(np.dot(np.transpose(y), s))) * np.dot(y, np.transpose(y)) - (1.0/(np.dot(np.dot(np.transpose(s), H), s))) * np.dot(np.dot(np.dot(H, s), np.transpose(s)), H)
    return Hnext

pprint(x)
pprint(pi)
xnew = np.zeros((x.shape[0]+1,1))
for i in range(x.shape[0]):
    xnew[i] = x[i]
xnew[x.shape[0]] = pi

pprint(xnew)

for iteration in range(maxIteration):
    xold = xnew
    gradFk = computeGrad(xold[:-1], xold[-1], expectedReturn, covarianceMatrix, delta)
    direction = np.dot(np.linalg.inv(H), gradFk)
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    #----------------------------------------------------------
    xnew = xold - stepSize * direction
    gradFk1 = computeGrad(xnew[:-1], xnew[-1], expectedReturn, covarianceMatrix, delta)
    s = xnew - xold
    y = gradFk1 - gradFk
    # BFGS Update
    Hnext = BFGSUpdate(H,y,s)
    H = Hnext
    
    f = computeF(xnew[:-1], expectedReturn, covarianceMatrix, delta)
    pprint(f)
    pprint(iteration)
    pprint(xold)
    pprint(H)
    pprint(gradFk)
    pprint(direction)
    pprint(xnew)
    pprint(gradFk1)
    pprint(y)
    pprint(s)